## Conceptos de aprendizaje  Bayesiano

Referencia: Machine Learning-a Probabilistic Approach Kevin Murphy 2012.

Considera la tarea de aprender un concepto en el siguiente sentido: a un aprendiz se le muestra una secuencia de instancias (objetos) etiquetados como pertenecientes a un concepto. Por ejemplo, a un niño se le muestra un animal y se le dice que es un *perro* o un *gato*. Los objetos pueden pertenecer a múltiples conceptos. Esperamos que el aprendiz pueda generalizar sobre la base de la descripción percibida de las instancias, de modo que las instancias no vistas puedan clasificarse en el conjunto de conceptos.

Modelamos esta tarea como la de aprender una función $f(instancia)$ mapeando instancias a valores booleanos - $f (x) = 1$ indica que $x$ pertenece al concepto representado por $f$, $f(x) = 0$ indica que $x$ no pertenece al concepto.

El siguiente ejemplo de esta tarea de aprendizaje de conceptos se denomina *number game* y fue introducido por Tenenbaum en 1999 en su tesis doctoral. En este juego, las instancias son un conjunto finito de números enteros (por ejemplo, $\{1,2,\dots, 100\}$). Los conceptos se describen mediante expresiones de lenguaje natural, por ejemplo, *números primos* o *números pares* o *potencias de 2*. Estos denotan cada uno un subconjunto de las instancias. Por ejemplo los *números pares* denota el subconjunto $\{2,4,6, \dots,98,100\}$.

El proceso de aprendizaje de conceptos se modela como una actualización incremental del modelo Bayesiano. El escenario del aprendizaje se ilustra con este ejemplo:

```
Quiero enseñarte un "concepto" que es un conjunto de números entre 1 y 100.
Te digo que "16" es un ejemplo positivo del concepto.
¿Qué otros números crees que son positivos?
¿Es "17"? "6"? "32"? "99"?

```

Naturalmente, con un solo ejemplo es difícil de responder, por lo que las predicciones serán vagas. Cuando decidamos la predicción, preferiremos números que sean "similares" a "16", pero ¿similares en qué sentido? "17" es similar porque está "cerca" de "16", "6" es similar porque tiene un dígito en común con "16", "32" porque también es par y también una potencia de "2", "99" no parece similar por ninguna "buena razón".

Por lo tanto, algunos números son más probables que otros y podemos representar nuestras predicciones como una distribución de probabilidad $p(\tilde{x}|D)$, donde $\tilde{x}$ es el número sobre el cual necesitamos hacer una predicción ("32" o "99") y $D$ es el conjunto de datos que hemos observado hasta ahora, inicialmente solo {"16"}. Esto se llama **distribución predictiva posterior**.

Podemos observar esta distribución predictiva posterior preguntando a un grupo de personas su predicción sobre todos los números entre $1$ y $100$ dados los ejemplos enseñados hasta ahora en forma de histograma. La siguiente figura (reproducida en Murphy 2012 desde el trabajo Tenenbaum 1999): muestra la distribución predictiva empírica promediada sobre las predicciones de $8$ humanos en el *number game*.


* Las dos primeras filas son las predicciones después de ver D = {16} y D = {60}. Esto ilustra una similitud difusa.
* La tercera fila después de ver D = {16,8,2,64}. Esto ilustra un comportamiento similar a una regla (potencias de 2).
* La última fila después de ver D = {16,23,19,20}. Esto ilustra una similitud enfocada (números cercanos a 20).

![](Number-game.png)

Suponga que observamos ejemplos positivos {"16", "8", "2", "64"} (como en la fila 3). La gente tiende a concluir que el concepto es el conjunto de "potencias de 2" (como se indica con los picos en el histograma que reflejan el juicio de $8$ personas). Este es un ejemplo de **inducción**.

Nuestra tarea es emular este comportamiento inductivo/de generalización utilizando un modelo matemático. El enfoque consiste en definir un espacio de hipótesis de conceptos, un conjunto $H$ de subconjuntos de números. Por ejemplo, números impares, números pares, todos los números, potencias de dos, números que terminan en $j$ ($j$ en $[0..9]$). El subconjunto de $H$ que es consistente con $D$ se denomina **espacio de versión**. A medida que obtenemos más ejemplos en $D$, el espacio de la versión se vuelve más pequeño y nos volvemos más seguros de nuestras predicciones.

Esta historia, sin embargo, no es suficiente para explicar el comportamiento de generalización observado. Vemos que preferimos algunos conceptos sobre otros (por ejemplo, dado $D = \{16,8,2,64\}$ preferimos "potencia de 2" sobre "números pares" o "todos los números"). Desarrollaremos un modelo Bayesiano para explicar estas preferencias.


### Likelihood (verosimilitud)

Debemos explicar por qué preferimos una hipótesis (potencia de dos) sobre otra dado el conjunto de datos $D$. La intuición es que queremos evitar coincidencias sospechosas: si el concepto verdadero es "números pares", ¿por qué solo vimos potencias de dos?

El modelo que adoptamos es: supongamos que muestreamos los datos observados en $D$ a partir de la extensión del concepto verdadero según una distribución uniforme. (La extensión de un concepto es el conjunto de instancias que pertenecen al concepto, por ejemplo $h_{par} = \{2,4,6,\dots, 98,100 \}$, la extensión de números que terminan en $9$ es $\{9, 19, \dots, 99 \}$) Ten en cuenta que esto es psicológicamente una suposición poco probable, porque sabemos que la gente prefiere "ejemplos típicos". 

Dada esta suposición, la probabilidad de muestrear $N$ elementos con reemplazo desde $h$ es:

$$p(D|h) = (1/\vert h\vert)^N \quad (donde N = \vert D\vert)$$

Esta ecuación crucial incorpora lo que Tenenbaum llama el principio del tamaño, lo que significa que el modelo favorece la hipótesis más simple (más pequeña) consistente con los datos. Esto se conoce más comúnmente como la navaja de Occam.

Por ejemplo:

$$p(\{16\}|h_{potencia2}) = 1/6 \quad  (\text{desde que}\ h_{potencia2}=\{2,4,8,16,32,64\} \text{tiene 6 elementos}), \quad p(\{16\} | h_{par}) = 1/50$$ 

Con $4$ ejemplos:

$$p(\{16,4,2,32\}|h_{potencia2}) = (1/6)^4 = 7.7\times 10^{-4}, \quad p(\{16,4,2,32\}|h_{par}) = (1/50)^4 = 1.6 \times 10^{-7}$$-

La tasa de verosimilitud es $5000:1$ en favor de $h_{potencia2}$. Esto cuantifica nuestra intuición anterior de que $D = \{16, 8, 2, 64\}$ sería una coincidencia muy sospechosa si se generara por $h_{par}$.

### Prior

En una perspectiva Bayesiana, queremos actualizar nuestras creencias anteriores dadas nuevas observaciones. Esto significa que también debemos modelar nuestras creencias previas sobre la probabilidad de los conceptos en el espacio de hipótesis.

Por ejemplo, consideramos $D = \{16,8,2,64\}$. Dado D, la hipótesis del concepto *h'= "potencias de 2 excepto 32"* es más probable que h = "potencias de 2" (dada la fórmula de probabilidad presentada).

*h'*, sin embargo, no parece un "concepto natural". Queremos modelar nuestra preferencia a priori por $h$ sobre $h'$. Esta clasificación de preferencias es difícil de justificar, se basa en criterios subjetivos. En este modelo, preparamos un espacio de hipótesis de unos $30$ conceptos y los clasificamos:

* Números pares
* Números impares
* Cuadrados
* Múltiplos de 3
* ...
* Múltiplos de 9
* Múltiplos de 10
* Termina en 1
* ...
* Termina en 9
* potencias de 2
* ...
* Potencias de 10
* Toda  
* Potencias de 2 + {37}
* Potencias de 2 - {32}

Damos una probabilidad previa más alta para los pares y los impares, todos los demás tienen la misma probabilidad, excepto los 2 últimos conceptos "no naturales" a los que asignamos una probabilidad muy baja.


![](Prior-posterior.png)

### Posterior

La distribución posterior $p(h|D)$ estima la probabilidad de cada concepto de hipótesis $h$ dada la observación D. Por la regla de Bayes, el posterior es proporcional a la verosimilitud multiplicada por el prior, normalizado por todos los posteriores:

$$p(h|D) = \frac{p(D|h)p(h)}{\sum_{h^{'} \in h}p(D, h^{'})}$$

Usando la definición de probabilidad anterior:

$$p(h|D) = \frac{p(h)I(D \in h)/\vert h\vert^N }{\sum_{h^{'} \in h}p(h^{'})I(D \in h^{'})/\vert h^{'} \vert^N}$$

donde $I(D \in h)$ es $1$ si todos los elementos en $D$ están en la extensión de la hipótesis $h$, $0$ en caso contrario.

La figura anterior dibuja el previo, la verosimilitud y el posterior después de ver $D = \{16\}$. Vemos que el posterior es una combinación del previo y verosimilitud. En el caso de la mayoría de los conceptos, el previo es uniforme, por lo que el posterior es proporcional a la verosimilitud.  Sin embargo, los conceptos como  "potencias de 2, más 37" y "potencias de 2, excepto 32" tienen un soporte posterior bajo, a pesar de tener una alta verosimilitud, debido al previo bajo. Por el contrario, el concepto de 'números impares' tiene un  posterior bajo, a pesar de tener un previo alto, debido a la baja verosimilitud.

En general, cuando tenemos suficientes datos, el posterior $p(h|D)$ alcanza su punto máximo en un solo concepto, a saber, la estimación MAP, es decir,

$$p(h|D) \rightarrow \delta_{\hat{h}^{MAP}}(h)$$


donde $\hat{h}^{MAP} = arg \max_{h}p(h|D)$ es el modo posterior y $\delta $ es la medida de Dirac, definida como:

$$\delta_x(A) = 1\ \text{si}\  x \in A; 0\ \text{si}\ x \notin A.$$


La estimación MAP puede ser escrita como:

$$\hat{h}^{MAP} = arg \max_{h}p(D|h)p(h) =  arg \max_{h}[\log p(D|h) + \log p(h)]$$


Dado que el término de probabilidad depende exponencialmente de $N$, y el previo permanece constante, a medida que obtenemos más y más datos, la estimación de MAP converge hacia la estimación de máxima verosimilitud o MLE:

$$\hat{h}^{MLE}= arg \max_{h}p(D|h)= arg \max_{h}\log p(D|h)$$


En otras palabras, si tenemos suficientes datos, vemos que los datos 'sobrepasa' a los previos. En este caso, la estimación de MAP converge hacia el MLE.

Si la hipótesis verdadera está en el espacio de hipótesis, entonces la estimación de MAP/ML convergerá en esta hipótesis. Por lo tanto, decimos que la inferencia bayesiana (y la estimación ML) son estimadores consistentes.

### Distribución predictiva posterior

El posterior es nuestro estado interno de creencias sobre el mundo. La forma de probar si nuestras creencias están justificadas es usarlas para predecir cantidades objetivamente observables (esta es la base del método científico). Específicamente, la distribución predictiva posterior en este contexto viene dada por:


$$p(\tilde{x} \in C|D) = \sum_{h}p(y =1|\tilde{x}, h)p(h|D)$$

Este es solo un promedio ponderado de las predicciones de cada hipótesis individual y se denomina promedio del modelo de Bayes.


### Un previo más complejo

Para modelar el comportamiento humano, Tenenbaum utilizó un previo ligeramente más sofisticado que se derivó del análisis de algunos datos experimentales de cómo las personas miden la similitud entre números. El resultado es un conjunto de conceptos aritméticos similares a los mencionados anteriormente, más todos los intervalos entre $n$ y $m$ para $1 \leq n, m \leq 100$. (Ten en cuenta que estas hipótesis no son mutuamente excluyentes). Por lo tanto, el previo es una mezcla de dos previos, uno sobre reglas aritméticas y uno sobre intervalos:

$$p(h) = \pi_{0}p_{reglas}(h) + (1- \pi_{0}) p_{intervalo}(h)$$

El único parámetro libre en el modelo es el peso relativo, $\pi_0$, dado a estas dos partes del previo. Los resultados no son muy sensibles a este valor, siempre que $\pi_0> 0.5$, lo que refleja el hecho de que es más probable que las personas piensen en conceptos definidos por reglas.

## Clasificador Naive Bayes

Naive Bayes es una técnica de clasificación estadística basada en el Teorema de Bayes. Es uno de los algoritmos de aprendizaje supervisado más simples. El clasificador Naive Bayes es un algoritmo rápido, preciso y confiable. Los clasificadores Naive Bayes tienen alta exactitud y velocidad en grandes conjuntos de datos.

El clasificador Naive Bayes asume que el efecto de una característica particular en una clase es independiente de otras características. Por ejemplo, un solicitante de préstamo es deseable o no dependiendo de sus ingresos, historial de préstamos y transacciones anteriores, edad y ubicación. Incluso si estas características son interdependientes, estas características aún se consideran de forma independiente. Esta suposición simplifica el cálculo y por eso se considera ingenua.

Referencia: [https://scikit-learn.org/stable/modules/naive_bayes.html](https://scikit-learn.org/stable/modules/naive_bayes.html).


### Ejemplo SMS

In [1]:
import pandas as pd
col_names = ['etiqueta', 'mensaje']
sms = pd.read_table('sms.tsv', sep='\t', header=None, names=col_names)
sms.shape

(5572, 2)

In [2]:
sms.etiqueta.value_counts()

ham     4825
spam     747
Name: etiqueta, dtype: int64

In [3]:
sms['etiqueta'] = sms.etiqueta.map({'ham':0, 'spam':1})

In [4]:
X = sms.mensaje
y = sms.etiqueta

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape
X_test.shape


(1393,)

### Vectorizando la data

`CountVectorizer` es una gran herramienta proporcionada por scikit-learn y se utiliza para transformar un texto dado en un vector sobre la base de la frecuencia de cada palabra que aparece en todo el texto a través de una matriz en la que cada palabra única está representada por una columna de la matriz  y cada muestra de texto del documento es una fila en la matriz. El valor de cada celda no es más que la frecuencia (conteo) de la palabra en esa muestra de texto en particular.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [7]:
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

<4179x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 55209 stored elements in Compressed Sparse Row format>

In [8]:
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm

<4179x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 55209 stored elements in Compressed Sparse Row format>

In [9]:
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1393x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 17604 stored elements in Compressed Sparse Row format>

###  Examinamos los tokens y los conteos

In [10]:
X_train_tokens = vect.get_feature_names()

In [11]:
X_train_tokens[:20]

['00',
 '000',
 '008704050406',
 '0121',
 '01223585236',
 '01223585334',
 '0125698789',
 '02',
 '0207',
 '02072069400',
 '02073162414',
 '02085076972',
 '021',
 '03',
 '04',
 '0430',
 '05',
 '050703',
 '0578',
 '06']

In [12]:
X_train_dtm.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Contamos cuántas veces aparece cada token en todos los mensajes en `X_train_dtm`.

In [13]:
import numpy as np
X_train_counts = np.sum(X_train_dtm.toarray(), axis=0)
X_train_counts

array([ 5, 23,  2, ...,  1,  1,  1], dtype=int64)

In [14]:
X_train_counts.shape

(7456,)

In [15]:
pd.DataFrame({'token':X_train_tokens, 'conteo':X_train_counts}).sort_values('conteo')

,token,conteo
3727,jules,1
4172,mallika,1
4169,malarky,1
4165,makiing,1
4161,maintaining,1
...,...,...
3502,in,683
929,and,717
6542,the,1004
7420,you,1660


### Calculamos el "spam" de cada token

In [16]:
sms_ham = sms[sms.etiqueta==0]
sms_spam = sms[sms.etiqueta==1]

In [17]:
vect.fit(sms.mensaje)
todos_tokens = vect.get_feature_names()

Creamos matrices de documentos para ham y spam.

In [18]:
ham_dtm = vect.transform(sms_ham.mensaje)
spam_dtm = vect.transform(sms_spam.mensaje)

In [19]:
ham_dtm

<4825x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 57836 stored elements in Compressed Sparse Row format>

In [20]:
ham_conteos = np.sum(ham_dtm.toarray(), axis=0)

In [21]:
spam_conteos = np.sum(spam_dtm.toarray(), axis=0)

In [22]:
token_conteos = pd.DataFrame({'token':todos_tokens, 'ham':ham_conteos, 'spam':spam_conteos})

In [23]:
token_conteos['ham'] = token_conteos.ham + 1
token_conteos['spam'] = token_conteos.spam + 1

Calculamos el radio de spam-ham para cada token.

In [24]:
token_conteos['radio_spam'] = token_conteos.spam/token_conteos.ham
token_conteos.sort_values('radio_spam')

,token,ham,spam,radio_spam
3684,gt,319,1,0.003135
4793,lt,317,1,0.003155
3805,he,232,1,0.004310
6843,she,168,1,0.005952
4747,lor,163,1,0.006135
...,...,...,...,...
369,18,1,52,52.000000
7837,tone,1,61,61.000000
352,150p,1,72,72.000000
6113,prize,1,94,94.000000


### Construyendo de un modelo Naive Bayes

El clasificador multinomial Naive Bayes es adecuado para la clasificación con características discretas (por ejemplo, el conteo de palabras para la clasificación de texto). La distribución multinomial normalmente requiere conteos de características enteras. Sin embargo, en la práctica, los conteos fraccionarios como **tf-idf** también pueden funcionar.

In [25]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

MultinomialNB()

In [41]:
y_pred_class = nb.predict(X_test_dtm)
print(sum(y_pred_class))

179


In [27]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.9885139985642498

La matriz de confusión es:

In [28]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[1203,    5],
       [  11,  174]], dtype=int64)

Predecimos las probabilidades (mal calibradas)

In [29]:
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([2.87744864e-03, 1.83488846e-05, 2.07301295e-03, ...,
       1.09026171e-06, 1.00000000e+00, 3.98279868e-09])

Calculamos AUC

In [30]:
metrics.roc_auc_score(y_test, y_pred_prob)

0.9866431000536962

Imprimimos el texto del mensaje para los falsos positivos

In [31]:
X_test[y_test < y_pred_class]

574               Waiting for your call.
3375             Also andros ice etc etc
45      No calls..messages..missed calls
3415             No pic. Please re-send.
1988    No calls..messages..missed calls
Name: mensaje, dtype: object

Imprimimos el texto del mensaje para los falsos negativos

In [32]:
X_test[y_test > y_pred_class]

3132    LookAtMe!: Thanks for your purchase of a video...
5       FreeMsg Hey there darling it's been 3 week's n...
3530    Xmas & New Years Eve tickets are now on sale f...
684     Hi I'm sue. I am 20 years old and work as a la...
1875    Would you like to see my XXX pics they are so ...
1893    CALL 09090900040 & LISTEN TO EXTREME DIRTY LIV...
4298    thesmszone.com lets you send free anonymous an...
4949    Hi this is Amy, we will be sending you a free ...
2821    INTERFLORA - It's not too late to order Inter...
2247    Hi ya babe x u 4goten bout me?' scammers getti...
4514    Money i have won wining number 946 wot do i do...
Name: mensaje, dtype: object

¿Qué notas sobre los falsos negativos?

In [33]:
# Tu solución

### Comparando el clasificador Naive Bayes con otros modelos

In [34]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train_dtm, y_train)

LogisticRegression(C=1000000000.0)

In [35]:
y_pred_class = logreg.predict(X_test_dtm)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

In [36]:
metrics.accuracy_score(y_test, y_pred_class)

0.9892318736539842

In [37]:
metrics.roc_auc_score(y_test, y_pred_prob)

0.9923035618399859

**Ejercicio:** Lee el código de [nltk.classify.naivebayes](http://www.nltk.org/_modules/nltk/classify/naivebayes.html) y ejecuta  `nltk.classify.naivebayes.demo()`.

In [45]:
# Tu solucion
# Natural Language Toolkit: Naive Bayes Classifiers
#
# Copyright (C) 2001-2020 NLTK Project
# Author: Edward Loper <edloper@gmail.com>
# URL: <http://nltk.org/>
# For license information, see LICENSE.TXT

"""
A classifier based on the Naive Bayes algorithm.  In order to find the
probability for a label, this algorithm first uses the Bayes rule to
express P(label|features) in terms of P(label) and P(features|label):

|                       P(label) * P(features|label)
|  P(label|features) = ------------------------------
|                              P(features)

The algorithm then makes the 'naive' assumption that all features are
independent, given the label:

|                       P(label) * P(f1|label) * ... * P(fn|label)
|  P(label|features) = --------------------------------------------
|                                         P(features)

Rather than computing P(features) explicitly, the algorithm just
calculates the numerator for each label, and normalizes them so they
sum to one:

|                       P(label) * P(f1|label) * ... * P(fn|label)
|  P(label|features) = --------------------------------------------
|                        SUM[l]( P(l) * P(f1|l) * ... * P(fn|l) )
"""

from collections import defaultdict

from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI
import nltk
nltk.download('names')

##//////////////////////////////////////////////////////
##  Naive Bayes Classifier
##//////////////////////////////////////////////////////


class NaiveBayesClassifier(ClassifierI):
    """
    A Naive Bayes classifier.  Naive Bayes classifiers are
    paramaterized by two probability distributions:

      - P(label) gives the probability that an input will receive each
        label, given no information about the input's features.

      - P(fname=fval|label) gives the probability that a given feature
        (fname) will receive a given value (fval), given that the
        label (label).

    If the classifier encounters an input with a feature that has
    never been seen with any label, then rather than assigning a
    probability of 0 to all labels, it will ignore that feature.

    The feature value 'None' is reserved for unseen feature values;
    you generally should not use 'None' as a feature value for one of
    your own features.
    """

    def __init__(self, label_probdist, feature_probdist):
        """
        :param label_probdist: P(label), the probability distribution
            over labels.  It is expressed as a ``ProbDistI`` whose
            samples are labels.  I.e., P(label) =
            ``label_probdist.prob(label)``.

        :param feature_probdist: P(fname=fval|label), the probability
            distribution for feature values, given labels.  It is
            expressed as a dictionary whose keys are ``(label, fname)``
            pairs and whose values are ``ProbDistI`` objects over feature
            values.  I.e., P(fname=fval|label) =
            ``feature_probdist[label,fname].prob(fval)``.  If a given
            ``(label,fname)`` is not a key in ``feature_probdist``, then
            it is assumed that the corresponding P(fname=fval|label)
            is 0 for all values of ``fval``.
        """
        self._label_probdist = label_probdist
        self._feature_probdist = feature_probdist
        self._labels = list(label_probdist.samples())

    def labels(self):
        return self._labels


    def classify(self, featureset):
        return self.prob_classify(featureset).max()


    def prob_classify(self, featureset):
        # Discard any feature names that we've never seen before.
        # Otherwise, we'll just assign a probability of 0 to
        # everything.
        featureset = featureset.copy()
        for fname in list(featureset.keys()):
            for label in self._labels:
                if (label, fname) in self._feature_probdist:
                    break
            else:
                # print('Ignoring unseen feature %s' % fname)
                del featureset[fname]

        # Find the log probabilty of each label, given the features.
        # Start with the log probability of the label itself.
        logprob = {}
        for label in self._labels:
            logprob[label] = self._label_probdist.logprob(label)

        # Then add in the log probability of features given labels.
        for label in self._labels:
            for (fname, fval) in featureset.items():
                if (label, fname) in self._feature_probdist:
                    feature_probs = self._feature_probdist[label, fname]
                    logprob[label] += feature_probs.logprob(fval)
                else:
                    # nb: This case will never come up if the
                    # classifier was created by
                    # NaiveBayesClassifier.train().
                    logprob[label] += sum_logs([])  # = -INF.

        return DictionaryProbDist(logprob, normalize=True, log=True)


    def show_most_informative_features(self, n=10):
        # Determine the most relevant features, and display them.
        cpdist = self._feature_probdist
        print("Most Informative Features")

        for (fname, fval) in self.most_informative_features(n):

            def labelprob(l):
                return cpdist[l, fname].prob(fval)

            labels = sorted(
                [l for l in self._labels if fval in cpdist[l, fname].samples()],
                key=lambda element: (-labelprob(element), element),
                reverse=True
            )
            if len(labels) == 1:
                continue
            l0 = labels[0]
            l1 = labels[-1]
            if cpdist[l0, fname].prob(fval) == 0:
                ratio = "INF"
            else:
                ratio = "%8.1f" % (
                    cpdist[l1, fname].prob(fval) / cpdist[l0, fname].prob(fval)
                )
            print(
                (
                    "%24s = %-14r %6s : %-6s = %s : 1.0"
                    % (fname, fval, ("%s" % l1)[:6], ("%s" % l0)[:6], ratio)
                )
            )


    def most_informative_features(self, n=100):
        """
        Return a list of the 'most informative' features used by this
        classifier.  For the purpose of this function, the
        informativeness of a feature ``(fname,fval)`` is equal to the
        highest value of P(fname=fval|label), for any label, divided by
        the lowest value of P(fname=fval|label), for any label:

        |  max[ P(fname=fval|label1) / P(fname=fval|label2) ]
        """
        if hasattr(self, "_most_informative_features"):
            return self._most_informative_features[:n]
        else:
            # The set of (fname, fval) pairs used by this classifier.
            features = set()
            # The max & min probability associated w/ each (fname, fval)
            # pair.  Maps (fname,fval) -> float.
            maxprob = defaultdict(lambda: 0.0)
            minprob = defaultdict(lambda: 1.0)

            for (label, fname), probdist in self._feature_probdist.items():
                for fval in probdist.samples():
                    feature = (fname, fval)
                    features.add(feature)
                    p = probdist.prob(fval)
                    maxprob[feature] = max(p, maxprob[feature])
                    minprob[feature] = min(p, minprob[feature])
                    if minprob[feature] == 0:
                        features.discard(feature)

            # Convert features to a list, & sort it by how informative
            # features are.
            self._most_informative_features = sorted(
                features, key=lambda feature_: (minprob[feature_] / maxprob[feature_], feature_[0],
                                                feature_[1] in [None, False, True], str(feature_[1]).lower())
            )
        return self._most_informative_features[:n]


    @classmethod
    def train(cls, labeled_featuresets, estimator=ELEProbDist):
        """
        :param labeled_featuresets: A list of classified featuresets,
            i.e., a list of tuples ``(featureset, label)``.
        """
        label_freqdist = FreqDist()
        feature_freqdist = defaultdict(FreqDist)
        feature_values = defaultdict(set)
        fnames = set()

        # Count up how many times each feature value occurred, given
        # the label and featurename.
        for featureset, label in labeled_featuresets:
            label_freqdist[label] += 1
            for fname, fval in featureset.items():
                # Increment freq(fval|label, fname)
                feature_freqdist[label, fname][fval] += 1
                # Record that fname can take the value fval.
                feature_values[fname].add(fval)
                # Keep a list of all feature names.
                fnames.add(fname)

        # If a feature didn't have a value given for an instance, then
        # we assume that it gets the implicit value 'None.'  This loop
        # counts up the number of 'missing' feature values for each
        # (label,fname) pair, and increments the count of the fval
        # 'None' by that amount.
        for label in label_freqdist:
            num_samples = label_freqdist[label]
            for fname in fnames:
                count = feature_freqdist[label, fname].N()
                # Only add a None key when necessary, i.e. if there are
                # any samples with feature 'fname' missing.
                if num_samples - count > 0:
                    feature_freqdist[label, fname][None] += num_samples - count
                    feature_values[fname].add(None)

        # Create the P(label) distribution
        label_probdist = estimator(label_freqdist)

        # Create the P(fval|label, fname) distribution
        feature_probdist = {}
        for ((label, fname), freqdist) in feature_freqdist.items():
            probdist = estimator(freqdist, bins=len(feature_values[fname]))
            feature_probdist[label, fname] = probdist

        return cls(label_probdist, feature_probdist)



##//////////////////////////////////////////////////////
##  Demo
##//////////////////////////////////////////////////////


def demo():
    from nltk.classify.util import names_demo

    classifier = names_demo(NaiveBayesClassifier.train)
    classifier.show_most_informative_features()





[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\HiroFoerYou\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\HiroFoerYou\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\HiroFoerYou\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\HiroFoerYou\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\HiroFoerYou\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\

In [46]:
demo()

Training classifier...
Testing classifier...
Accuracy: 0.7820
Avg. log likelihood: -0.7476

Unseen Names      P(Male)  P(Female)
----------------------------------------
  Kelli            0.0132  *0.9868
  Er              *0.8826   0.1174
  Ally             0.0903  *0.9097
  Stephan         *0.8361   0.1639
  Chriss           0.6864  *0.3136
Most Informative Features
                endswith = 'a'            female : male   =     31.5 : 1.0
                endswith = 'p'              male : female =     14.2 : 1.0
                endswith = 'v'              male : female =     13.0 : 1.0
                endswith = 'f'              male : female =     10.5 : 1.0
                endswith = 'm'              male : female =     10.3 : 1.0
                endswith = 'd'              male : female =     10.2 : 1.0
                endswith = 'o'              male : female =      7.7 : 1.0
                count(v) = 2              female : male   =      6.5 : 1.0
                endswith = 'r

**Ejercicio (opcional):** Lee el siguiente artículo de Andrew Y. Ng y Michael I. Jordan: [On Discriminative vs. Generative classifier: A comparison of logical regression and naive Bayes](http://ai.stanford.edu/~ang/papers/nips01-discriminativegenerative.pdf).

In [39]:
# Tu solucion